# [資料分析&機器學習] 第2.4講：資料前處理(Missing data, One-hot encoding, Feature Scaling)

參考資料:  
1. [[資料分析&機器學習] 第2.4講：資料前處理(Missing data, One-hot encoding, Feature Scaling) - JamesLearningNote - Medium](https://medium.com/jameslearningnote/%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90-%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%AC%AC2-4%E8%AC%9B-%E8%B3%87%E6%96%99%E5%89%8D%E8%99%95%E7%90%86-missing-data-one-hot-encoding-feature-scaling-3b70a7839b4a)
2. [standardize VS normalize - iT 邦幫忙::一起幫忙解決難題，拯救 IT 人的一天](https://ithelp.ithome.com.tw/articles/10276190)

資料的品質、特徵的選取決定了機器學習的上限，模型(Model)只是逼近這個上限。雖然在學術界總是以Model為主要討論對象，但實際上在業界80%的時間都是在對資料進行前處理，包含了資料獲取、清理、特徵選擇、特徵處理…到這裡我們可以稍微了解資料前處理的重要性。

當我們把資料輸入進Machine Learning的Model時，常常需要對資料先做一些前處理，提升Model的預測效果。主要是因為Machine learning的model是利用空間中的距離來做回歸或是分類，因此如果有缺值，就沒辦法在空間中表示出位置，所以需要補值。如果是類別資料用1, 2, 3來表示第1類、第2類、第3類會因為第3類比第1類數值還要大，在空間中會造成離原點距離較遠，因此需要使用one-hot encoding來處理。特徵縮放主要是可以幫助大部分的Model可以更快收斂並提升準確度。

常見的資料前處理如下所示：

1. 缺失值的處理
2. 類別資料的處理（有序、無序）
3. 資料特徵縮放

當然除了這三點之外，還有很多進階的資料前處理，比方說去掉一些不重要的欄位或是利用某些欄位去組合產生出新的欄位，都能夠提升Machine learning的預測準度。在資料前處理時，除了要對統計以及Model（像是Random forest 比起其他model少去很多資料前處理）知識有一定的了解之外，還必須具備有一定的產業知識(Domain Knowledge)，因為在不同產業的處理方式也會不太一樣，接下來就一一介紹吧

## 1. 缺失值的處理

缺值主要處理方式有兩種：

1. 丟棄，如果資料量夠多
2. 補值

常見補值方式有補固定值、平均值、眾數、中位數。較進階的補值方法有最近差補法（在資料中找與缺失樣本最接近的樣本來補足缺失的特徵）回歸方法(對帶有缺失的特徵根據其他的特徵屬性來建立回歸模型補值)，或是用插值法（拉格朗日插值法）

首先創立一些包含空值的csv並匯入Dataframe

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from io import StringIO

In [2]:
csv_data = '''A,B,C,D,E
           5.0,2.0,3.0,,6
           1.0,6.0,,8.0,5
           0.0,11.0,12.0,4.0,5
           3.0,,3.0,5.0,
           5.0,1.0,4.0,2.0,4
           '''

In [3]:
df = pd.read_csv(StringIO(csv_data))

In [4]:
df

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,NaN,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


<span style="color:gray">創立包含空值的Dataframe</span>

使用Pandas去空值的方法“dropna”，dropna預設只要任一欄位有空值，就會整筆刪掉。可透過參數來調整，像是把how設為all，就是要全部為空才清掉，或是用subset指定當某一欄為空時才刪。

In [5]:
df.dropna()

,A,B,C,D,E
2,0.0,11.0,12.0,4.0,5.0
4,5.0,1.0,4.0,2.0,4.0


In [6]:
df.dropna(how='all')

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,NaN,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


In [7]:
df.dropna(subset=['C'])

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,NaN,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


<span style="color:gray">pandas 去除空值的方法</span>

補值則使用fillna函式即可，依照以下範例即可補上固定值0、平均數、眾數、中位數…

In [8]:
df.fillna(0)

,A,B,C,D,E
0,5.0,2.0,3.0,0.0,6.0
1,1.0,6.0,0.0,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,0.0,3.0,5.0,0.0
4,5.0,1.0,4.0,2.0,4.0


In [9]:
df['B'] = df['B'].fillna(df['B'].mean())
df

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,5.0,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


In [10]:
df['C'] = df['C'].fillna(df['C'].mode())
df

,A,B,C,D,E
0,5.0,2.0,3.0,NaN,6.0
1,1.0,6.0,NaN,8.0,5.0
2,0.0,11.0,12.0,4.0,5.0
3,3.0,5.0,3.0,5.0,NaN
4,5.0,1.0,4.0,2.0,4.0


<span style="color:gray">Pandas補值方法：固定值0, 平均數, 眾數</span>

## 2. 類別資料的處理（有序、無序）

由於要在空間中表示點，所有的特徵都需要是數值，因此如果是類別的資料，像是XL,L,M,S,XS或是資料類別為Male, Female, Not Specified，就需要轉成數值才能在空間中來表示。有序的類別資料通常是直接使用數值替換，比方說XL, L, M, S, XS雖然是類別的屬性但因為有大小順序的關係，可以用10, 7 , 5, 3, 1來替換。如果是Male, Female, Not Specified因為這三種都是等價的關係因此需要找一個方法讓這三個屬性距離原點是相同距離，One-hot encoding 就是解決這的問題的方法，首先會將Male, Female, Not Specified由Gender從成一個欄位拆成三個欄位，因此編號1的使用者的屬性資料就是(1,0,0)編號2的使用者就是(0,1,0) 編號三個使用者就是(0,0,1)這三個使用者對於原點的距離都是1，就達成我們想要的結果了。但One-hot encoding的方法只適合類別種類少的形況下，如果類別種類太多就會產生出一大堆的特徵，造成其他的問題（比方說維數災難）。

![One-hot encoding 示意圖](1_MWciJw4Kwx_kaBSl0AbsRQ.webp)  
<span style="color:gray">One-hot encoding 示意圖</span>

首先先創立一個Dataframe，我們可以看到color跟size都是類別資料

### Cateforical Data(類別資料處理)

In [11]:
df2 = pd.DataFrame(
    [
        ['green', 'M', 10.1, 1],
        ['red', 'L', 13.5, 2],
        ['blue', 'XL', 15.3, 1]
    ]
)

df2.columns = ['color', 'size', 'price', 'classlabel']
df2

,color,size,price,classlabel
0,green,M,10.1,1
1,red,L,13.5,2
2,blue,XL,15.3,1


由於size是屬於有序的資料，我們只要稍微轉換為數值即可

In [12]:
size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1
}
df2['size'] = df2['size'].map(size_mapping)
df2

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,2
2,blue,3,15.3,1


<span style="color:gray">將有序類別資料轉為數值</span>

至於color我們就用onehot-encoding的方法來處理，在pandas裡面要使用onehot-encoding使用get_dummies這個函式就可以了，範例如下

In [13]:
pd.get_dummies(df2['color'])

,blue,green,red
0,0,1,0
1,0,0,1
2,1,0,0


In [14]:
onehot_encoding = pd.get_dummies(df2['color'], prefix='color')

In [15]:
df2 = df2.drop('color', 1)
df2

C:\Users\user\AppData\Local\Temp\ipykernel_17640\2083972832.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2 = df2.drop('color', 1)


,size,price,classlabel
0,1,10.1,1
1,2,13.5,2
2,3,15.3,1


In [16]:
pd.concat([onehot_encoding, df2], axis=1)

,color_blue,color_green,color_red,size,price,classlabel
0,0,1,0,1,10.1,1
1,0,0,1,2,13.5,2
2,1,0,0,3,15.3,1


<span style="color:gray">one-hot encoding在Pandas的用法吳恩達機器學習課程內的特徵縮放
</span>

## 3. 資料特徵縮放

In [20]:
iris = datasets.load_iris()
x = pd.DataFrame(iris['data'], columns=iris['feature_names'])
print("target_names: "+str(iris['target_names']))
y = pd.DataFrame(iris['target'], columns=['target_names'])
data = pd.concat([x,y], axis=1)
data.head(5)

target_names: ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


特徵縮放是資料前處理的一個很重要的關鍵，只少有部分的Model不需要做特徵縮放，像是決策樹以及隨機森林。需要特徵縮放主要是因為Model背後是用空間中的距離來做區分，假設某一個特徵過大，該Model的成本函數會被這個特徵所支配。另外在吳恩達的機器學習課程中有提到特徵縮放後能夠讓我們在做梯度下降時收斂更快。

舉例來說當某一個特徵範圍在零到兩千之間、另一個為一到五，當我們在做梯度下降時，整個等高線圖會呈現橢圓的形狀，因此收斂時沒辦法直接朝圓心(最低點)前進。若有更多興趣可參考吳恩達在Coursera上的課程。

![吳恩達機器學習](1_Ng4dEHk344oR_CD_yDy4cw.webp)  
<span style="color:gray">吳恩達機器學習課程內的特徵縮放</span>

簡單來說特徵縮放主要有兩種方法(這兩種常被混淆)：

##### 1. Normalization(據我了解中文翻譯還沒固定，有歸一化、常態化、區間縮放等等…正規化這個詞已經被解決overfitting 的 regularization拿去用了ＸＤ)

![Normalization](1_GQ1pZ4HP-Epa-0jkEkBAlg.webp)  
<span style="color:gray">Normalization</span>

最常見的Normalization為0–1區間縮放，經過Normalization之後資料的範圍會介在0~1之間，原本的最大值變為1，最小值變為0，具體作法如下

In [18]:
data['sepal length (cm)'] = (data['sepal length (cm)'] - data['sepal length (cm)'].min())/\
                            (data['sepal length (cm)'].max() - data['sepal length (cm)'].min())
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,0.222222,3.5,1.4,0.2,0
1,0.166667,3.0,1.4,0.2,0
2,0.111111,3.2,1.3,0.2,0
3,0.083333,3.1,1.5,0.2,0
4,0.194444,3.6,1.4,0.2,0


##### 2. Standardization(標準化)

![Standardization](1_Azz2Xb_FQVOCOhxFyzp0VA.webp)  
<span style="color:gray">Standardization</span>

經過Standardization資料的平均值會變為0, 標準差變為1，具體作法如下

In [19]:
data['sepal width (cm)'] = (data['sepal width (cm)'] - data['sepal width (cm)'].mean())/\
                           (data['sepal width (cm)'].std())
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,0.222222,1.015602,1.4,0.2,0
1,0.166667,-0.131539,1.4,0.2,0
2,0.111111,0.327318,1.3,0.2,0
3,0.083333,0.097889,1.5,0.2,0
4,0.194444,1.245030,1.4,0.2,0


對於新手來說這兩種選其中一種用即可。對於有經驗的同學，我稍微說明一下這兩種差異，經過Standardization之後，資料會符合常態分佈，不會有偏單邊的形況，由於常態分佈機器學習的加權迭代學習(梯度下降)可以更容易完成。另外Standardization還可以使離群值(outlier)對整個model的影響大大減低。